In [1]:
import os
os.chdir("../..")

In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix
import optuna

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
train_loader, val_loader, test_loader = load_datasets(
    data_dir="wildfire_dataset_scaled",
    batch_size=32,
    augmentation="baseline"
)

# Objective function for Optuna
def objective(trial):
    # Initialize VGG16 with last 2 layers unfrozen
    model = initialize_model("vgg16", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
    model.to(device)

    # Define hyperparameters to tune
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)

    # Set optimizer
    if optimizer_name == "Adam":
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=None,
        num_epochs=5,  # Use fewer epochs for hyperparameter search
        device=device
    )

    # Evaluate the model on validation data
    val_acc = history["val_acc"][-1]
    return val_acc

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Best hyperparameters
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

# Train final model with best hyperparameters
final_model = initialize_model("vgg16", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
final_model.to(device)

if best_params["optimizer"] == "Adam":
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, final_model.parameters()), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
elif best_params["optimizer"] == "SGD":
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, final_model.parameters()), lr=best_params["lr"], momentum=0.9, weight_decay=best_params["weight_decay"])
elif best_params["optimizer"] == "AdamW":
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, final_model.parameters()), lr=best_params["lr"], weight_decay=best_params["weight_decay"])

criterion = nn.CrossEntropyLoss()

# Train the final model
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,
    num_epochs=15,  # Full training with best parameters
    device=device
)

# Evaluate and visualize
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device)
plot_training(history, "outputs/tuned_training_curve.png")
plot_confusion_matrix(metrics["confusion_matrix"], ["No Fire", "Fire"], "outputs/tuned_confusion_matrix.png")


[I 2024-12-10 00:53:28,576] A new study created in memory with name: no-name-077e7243-2af1-4103-926c-7d7f44d33b22
c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Yash\AppData\Local\Temp\ipykernel_2816\4289293604.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com


Starting training...



Epoch [1/5]:
    Train Loss: 0.7438, Train Acc: 0.5941
    Val Loss:   0.6800, Val Acc:   0.6095
    Learning Rate: 0.000711



Epoch [2/5]:
    Train Loss: 0.6741, Train Acc: 0.6089
    Val Loss:   0.9303, Val Acc:   0.6119
    Learning Rate: 0.000711



Epoch [3/5]:
    Train Loss: 0.6797, Train Acc: 0.6460
    Val Loss:   0.5567, Val Acc:   0.7338
    Learning Rate: 0.000711



...

Epoch [1/5]:
    Train Loss: 0.4103, Train Acc: 0.8076
    Val Loss:   0.4026, Val Acc:   0.8308
    Learning Rate: 0.004076



Epoch [2/5]:
    Train Loss: 0.2492, Train Acc: 0.8956
    Val Loss:   0.2537, Val Acc:   0.9104
    Learning Rate: 0.004076



Epoch [3/5]:
    Train Loss: 0.1843, Train Acc: 0.9269
    Val Loss:   0.2541, Val Acc:   0.9055
    Learning Rate: 0.004076



Epoch [4/5]:
    Train Loss: 0.1163, Train Acc: 0.9587
    Val Loss:   0.3679, Val Acc:   0.8731
    Learning Rate: 0.004076



Epoch [5/5] - Training:  46%|████▌     | 27/59 [03:25<04:04,  7.63s/it, Batch Loss=0.0680]